# Session 2.3: Complete RAG System - End-to-End Implementation

## Overview

In this notebook, we'll build a complete RAG (Retrieval-Augmented Generation) system:
- Load and process documents (PDF, text files)
- Implement smart chunking strategies
- Build a complete RAG pipeline with ChromaDB
- Combine retrieval with LLM generation
- Compare RAG vs non-RAG answers
- Experiment with different configurations

**Key Concepts:**
- RAG combines retrieval (finding relevant information) with generation (LLM response)
- Proper chunking is critical for retrieval quality
- RAG reduces hallucinations by grounding answers in documents
- Different configurations affect accuracy, speed, and context quality

## The RAG Pipeline

```
INDEXING PHASE (offline, done once):
┌─────────────┐      ┌──────────┐      ┌────────────┐      ┌──────────────┐
│  Documents  │ ───> │ Chunking │ ───> │ Embeddings │ ───> │  Vector DB   │
│ (PDF, txt)  │      │ Strategy │      │ Generation │      │  (ChromaDB)  │
└─────────────┘      └──────────┘      └────────────┘      └──────────────┘

QUERY PHASE (online, for each query):
┌─────────┐      ┌────────────┐      ┌──────────────┐
│  Query  │ ───> │  Embed     │ ───> │  Similarity  │
└─────────┘      │  Query     │      │    Search    │
                 └────────────┘      └──────┬───────┘
                                            │
                                            ▼
                                     ┌──────────────┐
                                     │  Top-K       │
                                     │  Chunks      │
                                     └──────┬───────┘
                                            │
                                            ▼
┌──────────┐      ┌────────────┐      ┌──────────────┐
│  Answer  │ <─── │    LLM     │ <─── │   Prompt     │
│          │      │ Generation │      │ + Context    │
└──────────┘      └────────────┘      └──────────────┘
```

## Setup and Installation

In [ ]:
# Install required packages
!pip install chromadb openai pypdf2 requests -q

print("✓ Packages installed")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 82.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 102.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 19.0 MB/s 

In [ ]:
import chromadb
from chromadb.config import Settings
import json
from typing import List, Dict, Optional, Tuple
from openai import OpenAI
import os
import requests
import PyPDF2
import re
from pathlib import Path

print(f"ChromaDB version: {chromadb.__version__}")

ChromaDB version: 1.3.5


## Configure OpenRouter API

In [ ]:
# Add env variables from colab secrets
from google.colab import userdata
os.environ['OPENROUTER_API_KEY'] = userdata.get('OPENROUTER_API_KEY')

OPENROUTER_API_KEY = os.getenv('OPENROUTER_API_KEY', None)

if not OPENROUTER_API_KEY:
    raise ValueError("Please set your OpenRouter API key as an environment variable or directly in the code.")

# Initialize OpenAI client with OpenRouter endpoint
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY
)

# Models to use
EMBEDDING_MODEL = "openai/text-embedding-3-small"
LLM_MODEL = "openai/gpt-4o-mini"  # Fast and cost-effective for workshop

print(f"✓ OpenRouter client configured")
print(f"✓ Embedding model: {EMBEDDING_MODEL}")
print(f"✓ LLM model: {LLM_MODEL}")

✓ OpenRouter client configured
✓ Embedding model: openai/text-embedding-3-small
✓ LLM model: openai/gpt-4o-mini


## Part 1: Document Loading and Processing

### Why Document Processing Matters

Raw documents need preprocessing:
- **PDFs**: Extract text from pages
- **Cleaning**: Remove extra whitespace, formatting artifacts
- **Metadata**: Track source, page numbers for citations
- **Quality**: Ensure readable text for embeddings

In [ ]:
def load_pdf(file_path: str) -> Dict[str, any]:
    """
    Load PDF and extract text with metadata.

    Args:
        file_path: Path to PDF file

    Returns:
        Dictionary with text and metadata
    """
    with open(file_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)

        # Extract text from all pages
        full_text = ""
        page_texts = []

        for page_num, page in enumerate(pdf_reader.pages):
            page_text = page.extract_text()
            page_texts.append({
                "page_number": page_num + 1,
                "text": page_text
            })
            full_text += page_text + "\n"

        return {
            "source": file_path,
            "full_text": full_text,
            "page_texts": page_texts,
            "num_pages": len(pdf_reader.pages)
        }


def load_text_file(file_path: str) -> Dict[str, any]:
    """
    Load plain text file.

    Args:
        file_path: Path to text file

    Returns:
        Dictionary with text and metadata
    """
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

        return {
            "source": file_path,
            "full_text": text,
            "num_pages": 1
        }


def clean_text(text: str) -> str:
    """
    Clean extracted text.

    Args:
        text: Raw text

    Returns:
        Cleaned text
    """
    # Remove multiple newlines
    text = re.sub(r'\n\s*\n', '\n\n', text)

    # Remove excessive whitespace
    text = re.sub(r' +', ' ', text)

    # Remove page numbers (simple heuristic)
    text = re.sub(r'\n\d+\n', '\n', text)

    return text.strip()


print("✓ Document loading functions defined")

✓ Document loading functions defined


## Part 2: Chunking Strategies

### Why Chunking Matters

**The Problem:**
- Embedding models have token limits (usually 512-8192 tokens)
- Long documents lose semantic focus
- Need to split documents into smaller, meaningful pieces

**Chunking Strategies:**

1. **Fixed Size** - Simple, predictable size
2. **Sentence-based** - Maintains semantic coherence
3. **Sliding Window with Overlap** - Captures context across boundaries
4. **Semantic** - Split on topic changes (more complex)

**Key Parameters:**
- **Chunk size**: Too small = lacks context; Too large = dilutes relevance
- **Sweet spot**: 256-1024 tokens (roughly 200-800 words)
- **Overlap**: 10-20% helps capture context at boundaries

In [ ]:
def chunk_text_fixed(text: str, chunk_size: int = 500, overlap: int = 50) -> List[str]:
    """
    Chunk text using fixed word count with overlap.

    Args:
        text: Text to chunk
        chunk_size: Number of words per chunk
        overlap: Number of overlapping words between chunks

    Returns:
        List of text chunks
    """
    words = text.split()
    chunks = []

    for i in range(0, len(words), chunk_size - overlap):
        chunk = " ".join(words[i:i + chunk_size])
        if len(chunk.strip()) > 0:
            chunks.append(chunk)

    return chunks


def chunk_text_sentences(text: str, target_size: int = 500) -> List[str]:
    """
    Chunk text by sentences, grouping to approximate target size.

    Args:
        text: Text to chunk
        target_size: Target number of words per chunk

    Returns:
        List of text chunks
    """
    # Simple sentence splitting (naive, but works for most cases)
    sentences = re.split(r'(?<=[.!?])\s+', text)

    chunks = []
    current_chunk = []
    current_size = 0

    for sentence in sentences:
        sentence_size = len(sentence.split())

        if current_size + sentence_size > target_size and current_chunk:
            # Save current chunk and start new one
            chunks.append(" ".join(current_chunk))
            current_chunk = [sentence]
            current_size = sentence_size
        else:
            current_chunk.append(sentence)
            current_size += sentence_size

    # Add last chunk
    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks


def analyze_chunks(chunks: List[str]) -> None:
    """
    Analyze chunk statistics.

    Args:
        chunks: List of text chunks
    """
    word_counts = [len(chunk.split()) for chunk in chunks]
    char_counts = [len(chunk) for chunk in chunks]

    print(f"Number of chunks: {len(chunks)}")
    print(f"\nWord counts:")
    print(f"  Min: {min(word_counts)}")
    print(f"  Max: {max(word_counts)}")
    print(f"  Average: {sum(word_counts) / len(word_counts):.1f}")
    print(f"\nCharacter counts:")
    print(f"  Min: {min(char_counts)}")
    print(f"  Max: {max(char_counts)}")
    print(f"  Average: {sum(char_counts) / len(char_counts):.1f}")


print("✓ Chunking functions defined")

✓ Chunking functions defined


## Part 3: Creating Sample Documents

For this workshop, we'll create sample documents about AI and machine learning topics.

In a real scenario, you would load actual PDFs or documents.

In [ ]:
# Sample documents about AI/ML topics (in Croatian for consistency with previous notebooks)
sample_documents = [
    {
        "title": "Uvod u umjetnu inteligenciju",
        "content": """
Umjetna inteligencija (UI) je područje računarstva koje se bavi stvaranjem inteligentnih sustava
koji mogu učiti, rasuđivati i rješavati probleme. UI tehnologije uključuju strojno učenje,
duboko učenje i obrada prirodnog jezika.

Strojno učenje je podskup umjetne inteligencije gdje računala uče iz podataka bez eksplicitnog
programiranja. Algoritmi strojnog učenja prepoznaju uzorke u podacima i donose predviđanja ili odluke.

Postoje tri glavne vrste strojnog učenja: nadzirano učenje, nenadzirano učenje i učenje
potkrepljivanjem. Nadzirano učenje koristi označene podatke za treniranje modela. Nenadzirano
učenje pronalazi uzorke u neoznačenim podacima. Učenje potkrepljivanjem uči kroz interakciju s okolinom.
""",
        "category": "osnove",
        "language": "hr"
    },
    {
        "title": "Neuralne mreže i duboko učenje",
        "content": """
Neuralne mreže su računalni modeli inspirirani biološkim neuralnim mrežama. One se sastoje od
slojeva međusobno povezanih čvorova (neurona) koji obrađuju informacije.

Duboko učenje koristi neuralne mreže s više skrivenih slojeva. Svaki sloj uči sve kompleksnije
značajke iz podataka. Na primjer, u prepoznavanju slika, prvi sloj može naučiti rubove,
drugi sloj oblike, a dublji slojevi prepoznaju kompleksne objekte.

Backpropagation je algoritam koji se koristi za treniranje neuralnih mreža. On izračunava
gradijente funkcije gubitka korištenjem lančanog pravila i ažurira težine mreže kako bi
minimizirao grešku.

Konvolucijske neuralne mreže (CNN) su specijalizirane za obradu slika. Rekurzivne neuralne
mreže (RNN) su prikladne za sekvencijalne podatke poput teksta i govora.
""",
        "category": "duboko_ucenje",
        "language": "hr"
    },
    {
        "title": "Veliki jezični modeli (LLM)",
        "content": """
Veliki jezični modeli (Large Language Models - LLM) su neuralne mreže trenirane na ogromnim
količinama tekstualnih podataka. Oni mogu generirati tekst, prevoditi jezike, odgovarati na
pitanja i obavljati mnoge druge zadatke obrade prirodnog jezika.

Transformer arhitektura, predstavljena 2017. godine u radu "Attention is All You Need",
revolucionirala je obradu prirodnog jezika. Transformeri koriste mehanizam pažnje (attention)
koji omogućuje modelu da se fokusira na relevantne dijelove ulaza.

GPT (Generative Pre-trained Transformer) modeli su autoregressivni jezični modeli koji
predviđaju sljedeći token na temelju prethodnih tokena. BERT (Bidirectional Encoder
Representations from Transformers) koristi dvosmjerno kodiranje za bolje razumijevanje konteksta.

Prompt engineering je tehnika oblikovanja upita za LLM modele kako bi se dobili kvalitetniji
odgovori. Zero-shot i few-shot učenje omogućuju modelima da obavljaju zadatke s malo ili bez primjera.
""",
        "category": "llm",
        "language": "hr"
    },
    {
        "title": "RAG sustavi",
        "content": """
Retrieval-Augmented Generation (RAG) je tehnika koja kombinira pretraživanje dokumenata s
generiranjem teksta pomoću jezičnih modela. RAG sustavi smanjuju halucinacije i omogućuju
LLM modelima pristup ažurnim i specifičnim informacijama.

RAG se sastoji od dvije glavne faze: indeksiranja i upita. U fazi indeksiranja, dokumenti se
dijele na dijelove (chunks), pretvaraju u vektore (embeddings) i pohranjuju u vektorsku bazu podataka.

U fazi upita, korisnikov upit se pretvara u vektor i uspoređuje s vektorima dokumenata pomoću
kosinusne sličnosti. Najrelevantniji dijelovi dokumenata se dohvaćaju i šalju jezičnom modelu
zajedno s upitom.

Embedding modeli poput BERT-a ili text-embedding-3 pretvaraju tekst u numeričke vektore koji
hvataju semantičko značenje. Vektorske baze podataka poput ChromaDB, Pinecone ili Weaviate
omogućuju brzo pretraživanje sličnosti.

Strategije chunking-a (dijeljenja dokumenata) i odabir broja dohvaćenih dijelova (top-k)
značajno utječu na kvalitetu RAG sustava. Reranking i hibridno pretraživanje dodatno poboljšavaju rezultate.
""",
        "category": "rag",
        "language": "hr"
    },
    {
        "title": "Primjene umjetne inteligencije",
        "content": """
Umjetna inteligencija ima brojne praktične primjene u različitim industrijama. U zdravstvu,
AI pomaže u dijagnostici bolesti, analizi medicinskih slika i otkriću novih lijekova.

U financijskom sektoru, AI sustavi detektiraju prijevare, automatiziraju trgovanje i procjenjuju
kreditnu sposobnost. Chatbotovi i virtualni asistenti koriste NLP za komunikaciju s korisnicima.

Autonomna vozila koriste računalni vid, senzore i algoritme strojnog učenja za navigaciju i
donošenje odluka u realnom vremenu. Preporuke proizvoda i sadržaja na platformama poput
Netflixa i Amazona temelje se na sustavima preporuke.

U obrazovanju, AI omogućuje personalizirano učenje prilagođeno potrebama svakog učenika.
Sustavi za otkrivanje plagijata i automatsko ocjenjivanje olakšavaju rad nastavnicima.

Etička pitanja povezana s AI uključuju pristranost u podacima, privatnost, transparentnost
odluka i utjecaj na zapošljavanje. Odgovorna primjena AI zahtijeva pažljivo razmatranje
ovih pitanja.
""",
        "category": "primjene",
        "language": "hr"
    }
]

print(f"✓ Created {len(sample_documents)} sample documents")
print("\nDocument titles:")
for i, doc in enumerate(sample_documents, 1):
    print(f"  {i}. {doc['title']} ({doc['category']})")

✓ Created 5 sample documents

Document titles:
  1. Uvod u umjetnu inteligenciju (osnove)
  2. Neuralne mreže i duboko učenje (duboko_ucenje)
  3. Veliki jezični modeli (LLM) (llm)
  4. RAG sustavi (rag)
  5. Primjene umjetne inteligencije (primjene)


## Part 4: Document Indexing - Building the Vector Database

Now we'll process documents and store them in ChromaDB.

In [ ]:
class OpenRouterEmbeddingFunction:
    """
    Custom embedding function for ChromaDB using OpenRouter.
    """
    def __init__(self, api_key: str, model: str = EMBEDDING_MODEL):
        self.client = OpenAI(
            base_url="https://openrouter.ai/api/v1",
            api_key=api_key
        )
        self.model = model

    def __call__(self, input: List[str]) -> List[List[float]]:
        """
        Generate embeddings for a list of texts (for documents).
        ChromaDB expects this signature for adding documents.
        """
        response = self.client.embeddings.create(
            input=input,
            model=self.model
        )
        return [item.embedding for item in response.data]

    def embed_query(self, input: str) -> List[List[float]]: # Changed return type hint
        """
        Generate embedding for a single query text.
        ChromaDB expects this signature for queries.
        """
        response = self.client.embeddings.create(
            input=input,
            model=self.model
        )
        return [response.data[0].embedding] # Wrapped in a list to return List[List[float]]


# Create the embedding function instance
embedding_function = OpenRouterEmbeddingFunction(OPENROUTER_API_KEY)

print("✓ Embedding function created")

✓ Embedding function created


In [ ]:
import chromadb

# Set to True if you have to re-initialize the collection.
# Rerunning the below cell will throw an error if the collection already exists.
DELETE_THE_CHROMA_COLLECTION = True

if DELETE_THE_CHROMA_COLLECTION:
  try:
    # Initialize ChromaDB client (if not already done)
    client_db = chromadb.Client()

    # Delete the collection
    client_db.delete_collection(name="rag_documents")

    print("✓ 'rag_documents' collection deleted.")
  except Exception as e:
    print(f"Error deleting collection: {e}")

# Disable deleting the collection of the next run
DELETE_THE_CHROMA_COLLECTION = False

✓ 'rag_documents' collection deleted.


In [ ]:
# Initialize ChromaDB client
client_db = chromadb.Client()

# Create collection
collection = client_db.create_collection(
    name="rag_documents",
    embedding_function=embedding_function,
    metadata={
        "description": "RAG workshop document collection",
        # Chroma uses L2 distance by default, so we have to explicitly choose cosine distance
        "hnsw:space": "cosine",
    }
)

print(f"✓ Collection created: '{collection.name}'")

✓ Collection created: 'rag_documents'


In [ ]:
# Process and index documents
print("Processing and indexing documents...\n")

all_chunks = []
all_ids = []
all_metadata = []

chunk_id = 0

for doc_idx, doc in enumerate(sample_documents):
    print(f"Processing: {doc['title']}")

    # Clean text
    cleaned_text = clean_text(doc['content'])

    # Chunk the document
    chunks = chunk_text_fixed(cleaned_text, chunk_size=200, overlap=30)

    print(f"  Created {len(chunks)} chunks")

    # Add to lists
    for chunk_idx, chunk in enumerate(chunks):
        all_chunks.append(chunk)
        all_ids.append(f"doc{doc_idx}_chunk{chunk_idx}")
        all_metadata.append({
            "source": doc['title'],
            "category": doc['category'],
            "language": doc['language'],
            "chunk_index": chunk_idx,
            "total_chunks": len(chunks)
        })
        chunk_id += 1

print(f"\nTotal chunks created: {len(all_chunks)}")

# Add to ChromaDB
print("\nAdding to ChromaDB...")
collection.add(
    documents=all_chunks,
    ids=all_ids,
    metadatas=all_metadata
)

print(f"\n✓ Indexed {collection.count()} chunks in ChromaDB")
print(f"✓ RAG system ready!")

Processing and indexing documents...

Processing: Uvod u umjetnu inteligenciju
  Created 1 chunks
Processing: Neuralne mreže i duboko učenje
  Created 1 chunks
Processing: Veliki jezični modeli (LLM)
  Created 1 chunks
Processing: RAG sustavi
  Created 1 chunks
Processing: Primjene umjetne inteligencije
  Created 1 chunks

Total chunks created: 5

Adding to ChromaDB...

✓ Indexed 5 chunks in ChromaDB
✓ RAG system ready!


In [39]:
def inspect_chroma_query(question: str, collection, n_results: int = 5):
    """
    Queries ChromaDB and returns the raw output dictionary for inspection.

    Args:
        question: The query string.
        collection: The ChromaDB collection object.
        n_results: The number of results to retrieve.

    Returns:
        The raw dictionary returned by collection.query().
    """
    print(f"\nQuerying ChromaDB for: '{question}'")
    results = collection.query(
        query_texts=[question],
        n_results=n_results
    )
    return results

# Example usage:
sample_question = "Što su RAG sustavi?"
raw_chroma_output = inspect_chroma_query(sample_question, collection, n_results=3)

print("\n--- Raw ChromaDB Output ---")
print(json.dumps(raw_chroma_output, ensure_ascii=False, indent=2))
print("-------------------------")



Querying ChromaDB for: 'Što su RAG sustavi?'

--- Raw ChromaDB Output ---
{
  "ids": [
    [
      "doc3_chunk0",
      "doc1_chunk0",
      "doc4_chunk0"
    ]
  ],
  "embeddings": null,
  "documents": [
    [
      "Retrieval-Augmented Generation (RAG) je tehnika koja kombinira pretraživanje dokumenata s generiranjem teksta pomoću jezičnih modela. RAG sustavi smanjuju halucinacije i omogućuju LLM modelima pristup ažurnim i specifičnim informacijama. RAG se sastoji od dvije glavne faze: indeksiranja i upita. U fazi indeksiranja, dokumenti se dijele na dijelove (chunks), pretvaraju u vektore (embeddings) i pohranjuju u vektorsku bazu podataka. U fazi upita, korisnikov upit se pretvara u vektor i uspoređuje s vektorima dokumenata pomoću kosinusne sličnosti. Najrelevantniji dijelovi dokumenata se dohvaćaju i šalju jezičnom modelu zajedno s upitom. Embedding modeli poput BERT-a ili text-embedding-3 pretvaraju tekst u numeričke vektore koji hvataju semantičko značenje. Vektorske baze poda

## Part 5: The RAG Query Pipeline

Now we'll implement the complete query pipeline that:
1. Takes a user question
2. Retrieves relevant chunks from ChromaDB
3. Constructs a prompt with context
4. Generates an answer using an LLM

In [ ]:
def query_rag(question: str, collection, n_results: int = 5, model: str = LLM_MODEL,
              show_sources: bool = True) -> Dict[str, any]:
    """
    Complete RAG query pipeline.

    Args:
        question: User's question
        collection: ChromaDB collection
        n_results: Number of chunks to retrieve
        model: LLM model to use for generation
        show_sources: Whether to display retrieved sources

    Returns:
        Dictionary with answer and metadata
    """
    # Step 1: Retrieve relevant chunks
    results = collection.query(
        query_texts=[question],
        n_results=n_results
    )

    retrieved_chunks = results['documents'][0]
    metadatas = results['metadatas'][0]
    distances = results['distances'][0]

    if show_sources:
        print("\n" + "="*80)
        print("RETRIEVED SOURCES")
        print("="*80)
        for i, (chunk, meta, dist) in enumerate(zip(retrieved_chunks, metadatas, distances), 1):
            # Chroma returns cosine distance instead of cosine similarity, so we have to manually convert it back
            similarity = 1 - dist
            print(f"\n[{i}] Similarity: {similarity:.3f} | Source: {meta['source']} | Category: {meta['category']}")
            print(f"    {chunk[:200]}...")

    # Step 2: Construct prompt with context
    #chunk_list = []
    #for i, chunk in enumerate(retrieved_chunks):
    #    chunk_list.append(f"Dokument {i+1}: {chunk}")
    #context = "\n\n".join(chunk_list)

    context = "\n\n".join([
        f"[Dokument {i+1}]: {chunk}"
        for i, chunk in enumerate(retrieved_chunks)
    ])

    prompt = f"""Ti si pametan asistent koji odgovara na pitanja na temelju dostavljenog konteksta.

Kontekst:
{context}

Pitanje: {question}

Upute:
- Koristi SAMO informacije iz dostavljenog konteksta
- Ako odgovor nije u kontekstu, reci "Nemam tu informaciju u dostavljenim dokumentima"
- Citiraj koji dokument si koristio za svaku tvrdnju (npr. "Prema Dokumentu 1...")
- Budi koncizan ali potpun u odgovoru
- Odgovaraj na hrvatskom jeziku

Odgovor:"""

    # Step 3: Generate answer with LLM
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": prompt}
        ],
        temperature=0.3,  # Lower temperature for more factual responses
        max_tokens=1000
    )

    answer = response.choices[0].message.content

    return {
        "answer": answer,
        "sources": metadatas,
        "retrieved_chunks": retrieved_chunks,
        "similarities": [1 - d for d in distances]
    }


print("✓ RAG query function defined")

✓ RAG query function defined


## Part 6: Testing the RAG System

Let's test our RAG system with various questions.

In [ ]:
# Test Query 1: Technical definition
question1 = "Što je backpropagation i kako funkcionira?"

result1 = query_rag(question1, collection, n_results=3)

print("\n" + "="*80)
print("PITANJE:", question1)
print("="*80)
print("\nODGOVOR:")
print(result1["answer"])
print("\n" + "="*80)


RETRIEVED SOURCES

[1] Similarity: 0.631 | Source: Neuralne mreže i duboko učenje | Category: duboko_ucenje
    Neuralne mreže su računalni modeli inspirirani biološkim neuralnim mrežama. One se sastoje od slojeva međusobno povezanih čvorova (neurona) koji obrađuju informacije. Duboko učenje koristi neuralne mr...

[2] Similarity: 0.371 | Source: RAG sustavi | Category: rag
    Retrieval-Augmented Generation (RAG) je tehnika koja kombinira pretraživanje dokumenata s generiranjem teksta pomoću jezičnih modela. RAG sustavi smanjuju halucinacije i omogućuju LLM modelima pristup...

[3] Similarity: 0.364 | Source: Veliki jezični modeli (LLM) | Category: llm
    Veliki jezični modeli (Large Language Models - LLM) su neuralne mreže trenirane na ogromnim količinama tekstualnih podataka. Oni mogu generirati tekst, prevoditi jezike, odgovarati na pitanja i obavlj...

PITANJE: Što je backpropagation i kako funkcionira?

ODGOVOR:
Backpropagation je algoritam koji se koristi za treniranje neuraln

In [ ]:
# Test Query 2: Factual information
question2 = "Kada je predstavljena transformer arhitektura?"

result2 = query_rag(question2, collection, n_results=5)

print("\n" + "="*80)
print("PITANJE:", question2)
print("="*80)
print("\nODGOVOR:")
print(result2["answer"])
print("\n" + "="*80)


RETRIEVED SOURCES

[1] Similarity: 0.442 | Source: Veliki jezični modeli (LLM) | Category: llm
    Veliki jezični modeli (Large Language Models - LLM) su neuralne mreže trenirane na ogromnim količinama tekstualnih podataka. Oni mogu generirati tekst, prevoditi jezike, odgovarati na pitanja i obavlj...

[2] Similarity: 0.296 | Source: Neuralne mreže i duboko učenje | Category: duboko_ucenje
    Neuralne mreže su računalni modeli inspirirani biološkim neuralnim mrežama. One se sastoje od slojeva međusobno povezanih čvorova (neurona) koji obrađuju informacije. Duboko učenje koristi neuralne mr...

[3] Similarity: 0.289 | Source: RAG sustavi | Category: rag
    Retrieval-Augmented Generation (RAG) je tehnika koja kombinira pretraživanje dokumenata s generiranjem teksta pomoću jezičnih modela. RAG sustavi smanjuju halucinacije i omogućuju LLM modelima pristup...

[4] Similarity: 0.286 | Source: Uvod u umjetnu inteligenciju | Category: osnove
    Umjetna inteligencija (UI) je područje račun

In [ ]:
# Test Query 3: Conceptual explanation
question3 = "Objasni kako RAG sustavi smanjuju halucinacije?"

result3 = query_rag(question3, collection, n_results=5)

print("\n" + "="*80)
print("PITANJE:", question3)
print("="*80)
print("\nODGOVOR:")
print(result3["answer"])
print("\n" + "="*80)


RETRIEVED SOURCES

[1] Similarity: 0.488 | Source: RAG sustavi | Category: rag
    Retrieval-Augmented Generation (RAG) je tehnika koja kombinira pretraživanje dokumenata s generiranjem teksta pomoću jezičnih modela. RAG sustavi smanjuju halucinacije i omogućuju LLM modelima pristup...

[2] Similarity: 0.315 | Source: Primjene umjetne inteligencije | Category: primjene
    Umjetna inteligencija ima brojne praktične primjene u različitim industrijama. U zdravstvu, AI pomaže u dijagnostici bolesti, analizi medicinskih slika i otkriću novih lijekova. U financijskom sektoru...

[3] Similarity: 0.302 | Source: Neuralne mreže i duboko učenje | Category: duboko_ucenje
    Neuralne mreže su računalni modeli inspirirani biološkim neuralnim mrežama. One se sastoje od slojeva međusobno povezanih čvorova (neurona) koji obrađuju informacije. Duboko učenje koristi neuralne mr...

[4] Similarity: 0.275 | Source: Uvod u umjetnu inteligenciju | Category: osnove
    Umjetna inteligencija (UI) je područ

In [ ]:
# Test Query 4: Information not in documents
question4 = "Koliko parametara ima GPT-5?"

result4 = query_rag(question4, collection, n_results=5,model='mistralai/mistral-large-2512')

print("\n" + "="*80)
print("PITANJE:", question4)
print("="*80)
print("\nODGOVOR:")
print(result4["answer"])
print("\n" + "="*80)


RETRIEVED SOURCES

[1] Similarity: 0.384 | Source: Veliki jezični modeli (LLM) | Category: llm
    Veliki jezični modeli (Large Language Models - LLM) su neuralne mreže trenirane na ogromnim količinama tekstualnih podataka. Oni mogu generirati tekst, prevoditi jezike, odgovarati na pitanja i obavlj...

[2] Similarity: 0.300 | Source: RAG sustavi | Category: rag
    Retrieval-Augmented Generation (RAG) je tehnika koja kombinira pretraživanje dokumenata s generiranjem teksta pomoću jezičnih modela. RAG sustavi smanjuju halucinacije i omogućuju LLM modelima pristup...

[3] Similarity: 0.240 | Source: Primjene umjetne inteligencije | Category: primjene
    Umjetna inteligencija ima brojne praktične primjene u različitim industrijama. U zdravstvu, AI pomaže u dijagnostici bolesti, analizi medicinskih slika i otkriću novih lijekova. U financijskom sektoru...

[4] Similarity: 0.227 | Source: Uvod u umjetnu inteligenciju | Category: osnove
    Umjetna inteligencija (UI) je područje računarstv

## Part 7: RAG vs Non-RAG Comparison

Let's compare answers with and without RAG to see the difference.

In [ ]:
def query_without_rag(question: str, model: str = LLM_MODEL) -> str:
    """
    Query LLM directly without RAG context.

    Args:
        question: User's question
        model: LLM model to use

    Returns:
        Generated answer
    """

    prompt = f"""Ti si pametan asistent koji odgovara na pitanja.

Pitanje: {question}

Upute:
- Ako ne znaš odgovor ili nemaš dovoljno informacija, reci da ne znaš.
- Budi koncizan ali potpun u odgovoru.
- Odgovaraj na hrvatskom jeziku.

Odgovor:"""

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": prompt}
        ],
        temperature=0.3,
        max_tokens=1000
    )

    return response.choices[0].message.content


print("✓ Non-RAG query function defined")

✓ Non-RAG query function defined


In [ ]:
# Compare on a specific question
comparison_question = "Koliko parametara ima GPT-5 model"
TEST_MODEL  = 'qwen/qwen3-vl-30b-a3b-thinking'

print("\n" + "="*80)
print("PITANJE:", comparison_question)
print("MODEL:  ", TEST_MODEL)
print("="*80)


# Get RAG answer
rag_answer = query_rag(comparison_question, collection, n_results=3, show_sources=False, model=TEST_MODEL)

# Get non-RAG answer
no_rag_answer = query_without_rag(comparison_question, model=TEST_MODEL)

print("\n" + "-"*80)
print("SA RAG-om (s kontekstom iz dokumenata):")
print("-"*80)
print(rag_answer["answer"])

print("\n" + "-"*80)
print("BEZ RAG-a (samo znanje modela):")
print("-"*80)
print(no_rag_answer)

print("\n" + "="*80)
print("ANALIZA:")
print("="*80)
print("""
RAG prednosti:
✓ Temelji odgovor na specifičnim dokumentima
✓ Može citirati izvore
✓ Smanjuje halucinacije
✓ Omogućuje ažuriranje znanja bez ponovnog treniranja modela

Bez RAG-a:
✗ Ovisi samo o znanju iz treninga
✗ Može generirati netočne informacije
✗ Nema izvora za provjeru
✓ Ali može biti brži za opća pitanja
""")


PITANJE: Koliko parametara ima GPT-5 model
MODEL:   qwen/qwen3-vl-30b-a3b-thinking

--------------------------------------------------------------------------------
SA RAG-om (s kontekstom iz dokumenata):
--------------------------------------------------------------------------------


Nemam tu informaciju u dostavljenim dokumentima. 

Ni u jednom od tri dokumenta (Dokument 1, Dokument 2 ili Dokument 3) se ne spominje broj parametara za GPT-5 model. Dokumenti opisuju opće koncepte velikih jezičnih modela, Transformer arhitekture i RAG tehnologiju, ali ne sadrže specifične informacije o broju parametara za GPT-5.

--------------------------------------------------------------------------------
BEZ RAG-a (samo znanje modela):
--------------------------------------------------------------------------------


GPT-5 model još nije objavljen, pa nema dostupnih informacija o broju parametara. Trenutno najnoviji model je GPT-4.

ANALIZA:

RAG prednosti:
✓ Temelji odgovor na specifičnim dokum

## Part 8: Experiments - Optimizing RAG Configuration

Let's experiment with different RAG configurations.

In [ ]:
# Experiment 1: Varying number of retrieved chunks (top-k)

test_question = "Što je RAG i kako funkcionira?"

print("EKSPERIMENT 1: Usporedba različitih top-k vrijednosti")
print("="*80)

top_k_values = [3, 5, 10]

for k in top_k_values:
    print(f"\n{'='*80}")
    print(f"Testiranje s top-{k} rezultata")
    print('='*80)

    result = query_rag(test_question, collection, n_results=k, show_sources=False)

    print(f"\nOdgovor:")
    print(result["answer"])

    print(f"\nBroj dohvaćenih izvora: {len(result['sources'])}")
    print(f"Prosječna sličnost: {sum(result['similarities']) / len(result['similarities']):.3f}")

EKSPERIMENT 1: Usporedba različitih top-k vrijednosti

Testiranje s top-3 rezultata

Odgovor:
RAG (Retrieval-Augmented Generation) je tehnika koja kombinira pretraživanje dokumenata s generiranjem teksta pomoću jezičnih modela. Funkcionira u dvije glavne faze: indeksiranja i upita. U fazi indeksiranja, dokumenti se dijele na dijelove (chunks), pretvaraju u vektore (embeddings) i pohranjuju u vektorsku bazu podataka. U fazi upita, korisnikov upit se pretvara u vektor i uspoređuje s vektorima dokumenata pomoću kosinusne sličnosti. Najrelevantniji dijelovi dokumenata se dohvaćaju i šalju jezičnom modelu zajedno s upitom. Embedding modeli poput BERT-a ili text-embedding-3 koriste se za pretvaranje teksta u numeričke vektore koji hvataju semantičko značenje. Vektorske baze podataka poput ChromaDB, Pinecone ili Weaviate omogućuju brzo pretraživanje sličnosti. Kvalitetu RAG sustava značajno utječu strategije chunking-a i odabir broja dohvaćenih dijelova (top-k) (prema Dokumentu 1).

Broj dohv

In [ ]:
# Experiment 2: Metadata filtering

print("\nEKSPERIMENT 2: Pretraživanje s filterima")
print("="*80)

question = "Objasni duboko učenje"

# Query with category filter
filtered_results = collection.query(
    query_texts=[question],
    n_results=5,
    where={"category": "duboko_ucenje"}  # Filter by category
)

print(f"\nPitanje: {question}")
print(f"Filter: category='duboko_ucenje'")
print("\nDohvaćeni dokumenti:")

for i, (doc, meta, dist) in enumerate(
    zip(
        filtered_results['documents'][0],
        filtered_results['metadatas'][0],
        filtered_results['distances'][0]
    ), 1
):
    print(f"\n[{i}] Izvor: {meta['source']} | Kategorija: {meta['category']} | Sličnost: {1-dist:.3f}")
    print(f"    {doc[:150]}...")


EKSPERIMENT 2: Pretraživanje s filterima

Pitanje: Objasni duboko učenje
Filter: category='duboko_ucenje'

Dohvaćeni dokumenti:

[1] Izvor: Neuralne mreže i duboko učenje | Kategorija: duboko_ucenje | Sličnost: 0.437
    Neuralne mreže su računalni modeli inspirirani biološkim neuralnim mrežama. One se sastoje od slojeva međusobno povezanih čvorova (neurona) koji obrađ...


In [ ]:
# Experiment 3: Answer quality with confidence thresholds

def query_rag_with_confidence(question: str, collection, threshold: float = 0.7,
                               n_results: int = 5) -> Dict[str, any]:
    """
    RAG query with confidence threshold - only answer if relevance is high enough.

    Args:
        question: User's question
        collection: ChromaDB collection
        threshold: Minimum similarity threshold (0-1)
        n_results: Number of chunks to retrieve

    Returns:
        Dictionary with answer and confidence info
    """
    results = collection.query(
        query_texts=[question],
        n_results=n_results
    )

    distances = results['distances'][0]
    similarities = [1 - d for d in distances]
    max_similarity = max(similarities)

    if max_similarity < threshold:
        return {
            "answer": f"⚠️ Pouzdanost odgovora je preniska (max sličnost: {max_similarity:.3f}). Nemam dovoljno relevantnih informacija za odgovor na ovo pitanje.",
            "confidence": max_similarity,
            "answered": False
        }

    # Proceed with normal RAG if confidence is sufficient
    result = query_rag(question, collection, n_results=n_results, show_sources=False)
    result["confidence"] = max_similarity
    result["answered"] = True

    return result


print("EKSPERIMENT 3: Odgovori s pragom pouzdanosti")
print("="*80)

# Test with relevant question
relevant_q = "Što je transformer arhitektura?"
result_relevant = query_rag_with_confidence(relevant_q, collection, threshold=0.6)

print(f"\nPitanje: {relevant_q}")
print(f"Pouzdanost: {result_relevant['confidence']:.3f}")
print(f"Odgovoreno: {result_relevant['answered']}")
print(f"\nOdgovor:\n{result_relevant['answer'][:300]}...")

# Test with irrelevant question
print("\n" + "-"*80)
irrelevant_q = "Koja je najbolja pizza u Zagrebu?"
result_irrelevant = query_rag_with_confidence(irrelevant_q, collection, threshold=0.6)

print(f"\nPitanje: {irrelevant_q}")
print(f"Pouzdanost: {result_irrelevant['confidence']:.3f}")
print(f"Odgovoreno: {result_irrelevant['answered']}")
print(f"\nOdgovor:\n{result_irrelevant['answer']}")

EKSPERIMENT 3: Odgovori s pragom pouzdanosti

Pitanje: Što je transformer arhitektura?
Pouzdanost: 0.400
Odgovoreno: False

Odgovor:
⚠️ Pouzdanost odgovora je preniska (max sličnost: 0.400). Nemam dovoljno relevantnih informacija za odgovor na ovo pitanje....

--------------------------------------------------------------------------------

Pitanje: Koja je najbolja pizza u Zagrebu?
Pouzdanost: 0.198
Odgovoreno: False

Odgovor:
⚠️ Pouzdanost odgovora je preniska (max sličnost: 0.198). Nemam dovoljno relevantnih informacija za odgovor na ovo pitanje.


## Part 9: Interactive RAG Demo

Try your own questions!

### Analiza Eksperimenta 3: Odgovori s Pragom Pouzdanosti

U Eksperimentu 3 testirali smo funkcionalnost `query_rag_with_confidence` koja koristi prag sličnosti za procjenu pouzdanosti odgovora. Prag je postavljen na `0.6`.

- **Relevantno pitanje ("Što je transformer arhitektura?")**: Maksimalna sličnost dohvaćenih dokumenata bila je `0.400`. Budući da je ova vrijednost niža od praga od `0.6`, sustav je ispravno prepoznao da nema dovoljno relevantnih informacija i vratio poruku o niskoj pouzdanosti, umjesto da pokuša generirati odgovor koji bi mogao biti netočan ili haluciniran.

- **Irelevantno pitanje ("Koja je najbolja pizza u Zagrebu?")**: Očekivano, za ovo pitanje je maksimalna sličnost bila vrlo niska (`0.198`), daleko ispod praga, što je rezultiralo istom porukom o niskoj pouzdanosti. Ovo demonstrira kako prag pomaže sustavu da prepozna kada pitanje nije pokriveno njegovim znanjem.

**Zaključak:** Postavljanje praga pouzdanosti ključno je za poboljšanje robusnosti RAG sustava, jer sprječava davanje odgovora kada je relevantnost dohvaćenih informacija upitna. Međutim, važno je kalibrirati prag specifično za korišteni embedding model i domenu, budući da različiti modeli mogu proizvoditi različite distribucije sličnosti.

In [ ]:
# Interactive query - change this to any question you want
your_question = "Koje su primjene umjetne inteligencije u zdravstvu?"

result = query_rag(your_question, collection, n_results=2, model='deepseek/deepseek-v3.2-speciale')

print("\n" + "="*80)
print("PITANJE:", your_question)
print("="*80)
print("\nODGOVOR:")
print(result["answer"])
print("\n" + "="*80)


RETRIEVED SOURCES

[1] Similarity: 0.708 | Source: Primjene umjetne inteligencije | Category: primjene
    Umjetna inteligencija ima brojne praktične primjene u različitim industrijama. U zdravstvu, AI pomaže u dijagnostici bolesti, analizi medicinskih slika i otkriću novih lijekova. U financijskom sektoru...

[2] Similarity: 0.613 | Source: Uvod u umjetnu inteligenciju | Category: osnove
    Umjetna inteligencija (UI) je područje računarstva koje se bavi stvaranjem inteligentnih sustava koji mogu učiti, rasuđivati i rješavati probleme. UI tehnologije uključuju strojno učenje, duboko učenj...

PITANJE: Koje su primjene umjetne inteligencije u zdravstvu?

ODGOVOR:
Prema Dokumentu 1, umjetna inteligencija u zdravstvu pomaže u dijagnostici bolesti, analizi medicinskih slika i otkriću novih lijekova.



## Summary and Key Takeaways

### What We Built

1. **Complete RAG Pipeline**
   - Document loading and processing
   - Smart chunking strategies
   - Vector storage with ChromaDB
   - Retrieval + Generation workflow

2. **Key Components**
   - **Indexing**: Documents → Chunks → Embeddings → Vector DB
   - **Retrieval**: Query → Embedding → Similarity Search → Top-K chunks
   - **Generation**: Context + Query → LLM → Answer

3. **Configuration Parameters**
   - Chunk size and overlap
   - Number of retrieved chunks (top-k)
   - Similarity threshold
   - LLM temperature

### RAG Benefits

✅ **Reduces hallucinations** - Grounds answers in documents  
✅ **Source attribution** - Can cite specific documents  
✅ **Up-to-date information** - Add new docs without retraining  
✅ **Domain-specific** - Works with specialized knowledge  
✅ **Controllable** - Filter by metadata, adjust parameters  

### When to Use RAG

- ✓ Domain-specific Q&A systems
- ✓ Document search and analysis
- ✓ Customer support with knowledge bases
- ✓ Research assistants
- ✓ Compliance and legal document queries

### Best Practices

1. **Chunking**: 256-1024 tokens, 10-20% overlap
2. **Retrieval**: Start with top-5, adjust based on needs
3. **Prompts**: Be explicit about using only context
4. **Evaluation**: Test with diverse questions
5. **Thresholds**: Set confidence thresholds for production

### Next Steps

In Session 3, we'll learn:
- **Reranking** for better retrieval precision
- **Hybrid search** combining semantic + keyword search
- **Proper evaluation** methods (LLM-as-judge)
- **Why cosine similarity fails** for answer evaluation
- **Production optimization** techniques

## Exercises (Optional)

Try these to deepen your understanding:

1. **Add your own documents**: Load PDFs or text files and index them
2. **Experiment with chunk sizes**: Try 128, 256, 512, 1024 - which works best?
3. **Test different embedding models**: Compare text-embedding-3-small vs large
4. **Implement semantic chunking**: Split on section headers or topics
5. **Add source citations**: Format answers with clickable source links
6. **Build a comparison tool**: Evaluate multiple questions systematically
7. **Try different LLMs**: Compare GPT-4o-mini vs other models for generation

In [ ]:
# Your experiments here!
